In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

In [ ]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 1297 (delta 32), reused 32 (delta 15), pack-reused 1238
Receiving objects: 100% (1297/1297), 3.54 MiB | 73.00 KiB/s, done.
Resolving deltas: 100% (862/862), done.


In [ ]:
!mv yolov5/* ./

In [ ]:
!pip install -r requirements.txt

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-zsanlcja
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-zsanlcja
     |████████████████████████████████| 20.4MB 1.4MB/s 
     |████████████████████████████████| 276kB 53.8MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=a20b7928d5dc0e84ed3b400a5599db41e281688d99cdfb4fd8ecef25da668165
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Created wheel for pycocotools: filename=pycocotools-2.0-cp36-cp36m-linux_x86_64.whl size=266989 sha256=25b6a4a0a114ee8ed90a058297a27c9921563021807eb0805afd9732c126a1d9
  Stored in directory: /tmp/pip-ephem-wheel-cache-a9v8kmrp/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
Successfully built PyYAML pycocotools
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incom

In [ ]:
df = pd.read_csv('/content/drive/My Drive/ANPD/train.csv')
df.columns =['image_id','x','y','w','h','classes']
df['w'] = df['w'] - df['x']
df['h'] = df['h'] - df['y']
df['x_center'] = df['x'] + df['w']/2
df['y_center'] = df['y'] + df['h']/2
df['classes'] = 0
from tqdm.auto import tqdm
import shutil as sh
df = df[['image_id','x', 'y', 'w', 'h','x_center','y_center','classes']]

In [ ]:
df.head()

,image_id,x,y,w,h,x_center,y_center,classes
0,LA03304_bg,55,43,160,82,135.0,84.0,0
1,LA10214_bg,154,113,163,130,235.5,178.0,0
2,LA26527_bg,46,151,104,159,98.0,230.5,0
3,LA28823_bg,30,4,217,112,138.5,60.0,0
4,LA29127_bg,31,124,122,191,92.0,219.5,0


In [ ]:
index = list(set(df.image_id))

In [ ]:
import shutil
shutil.rmtree('/content/convertor/fold0')

In [ ]:
source = 'train'
if True:
    for fold in [0]:
        val_index = index[len(index)*fold//5:len(index)*(fold+1)//5]
        for name,mini in tqdm(df.groupby('image_id')):
            if name in val_index:
                path2save = 'val2017/'
            else:
                path2save = 'train2017/'
            if not os.path.exists('convertor/fold{}/labels/'.format(fold)+path2save):
                os.makedirs('convertor/fold{}/labels/'.format(fold)+path2save)
            with open('convertor/fold{}/labels/'.format(fold)+path2save+name+".txt", 'w+') as f:
                row = mini[['classes','x_center','y_center','w','h']].astype(float).values
                row = row/600
                row = row.astype(str)
                for j in range(len(row)):
                    text = ' '.join(row[j])
                    f.write(text)
                    f.write("\n")
            if not os.path.exists('convertor/fold{}/images/{}'.format(fold,path2save)):
                os.makedirs('convertor/fold{}/images/{}'.format(fold,path2save))
            sh.copy("/content/drive/My Drive/ANPD/{}/{}.jpg".format(source,name),'convertor/fold{}/images/{}/{}.jpg'.format(fold,path2save,name))

In [ ]:
!python train.py --noautoanchor --img 608 --batch 8 --epochs 300 --data "/content/drive/My Drive/ANPD/configYolo5/wheat0.yaml" --cfg "/content/drive/My Drive/ANPD/configYolo5/yolov5x.yaml" --name yolov5x_fold0

Apex recommended for faster mixed precision training: https://github.com/NVIDIA/apex
{'lr0': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'giou': 0.05, 'cls': 0.58, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0, 'hsv_h': 0.014, 'hsv_s': 0.68, 'hsv_v': 0.36, 'degrees': 0.0, 'translate': 0.0, 'scale': 0.5, 'shear': 0.0}
Namespace(adam=False, batch_size=8, bucket='', cache_images=False, cfg='/content/drive/My Drive/ANPD/configYolo5/yolov5x.yaml', data='/content/drive/My Drive/ANPD/configYolo5/wheat0.yaml', device='', epochs=300, evolve=False, img_size=[608], multi_scale=False, name='yolov5x_fold0', noautoanchor=True, nosave=False, notest=False, rect=False, resume=False, single_cls=False, weights='')
Using CUDA device0 _CudaDeviceProperties(name='Tesla K80', total_memory=11441MB)

2020-06-29 16:51:31.987479: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Start Tensorboard w

In [ ]:
ls

convertor/  Dockerfile  inference/  README.md*         test.py         utils/
data/       drive/      LICENSE     requirements.txt*  train.py        weights/
detect.py   hubconf.py  models/     sample_data/       tutorial.ipynb  yolov5/
